In [48]:
import pandas as pd
import numpy as np
from pickle import load as load_model
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [47]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')
data_train.shape,data_test.shape

((891, 12), (418, 11))

##### Load Models

In [9]:
gender_encoder = load_model(open('gender_encoder.sav','rb'))
embarked_encoder = load_model(open('embarked_encoder.sav','rb'))
model = load_model(open('RandomForestClassifier.sav','rb'))

In [35]:
def preprocess(data):
    features = ['Pclass','Sex','SibSp','Parch','Embarked','Age']
    data  = data[features]
    data['Age'] = data['Age'].fillna('0')
    data['Embarked'] = data['Embarked'].fillna('NA')
    data[gender_encoder.categories_[0][-1]] = gender_encoder.transform(np.array(data['Sex']).reshape((data.shape[0],-1))).reshape(data.shape[0])
    data[embarked_encoder.categories_[0]] = embarked_encoder.transform(np.array(data['Embarked']).reshape((data.shape[0],-1)))
    data.drop('Sex',inplace=True,axis=1)
    data.drop('Embarked',inplace=True,axis=1)
    return data

#### Train Evaluation

In [49]:
x_train = preprocess(data_train)
y_train = data_train['Survived']
y_pred_train = model.predict(x_train)

acc = accuracy_score(y_train,y_pred_train)
prc = precision_score(y_train,y_pred_train)
rcl = recall_score(y_train,y_pred_train)

print(acc,prc,rcl)

confusion_matrix(y_train,y_pred_train)


0.9371492704826038 0.946875 0.8859649122807017


array([[532,  17],
       [ 39, 303]], dtype=int64)

#### Test Prediction

In [45]:
x_test = preprocess(data_test)
y_pred = model.predict(x_test)
pred_results = pd.concat([data_test['PassengerId'],pd.DataFrame(y_pred,columns=['Survived'])],axis=1)
pred_results.to_csv('kaggle_submission.csv',index=False)